In [5]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys
sys.path.append('/app')

In [2]:
import uuid
from datetime import datetime
from pytz import timezone
from psql_models import User, UserToken, async_session


In [3]:
from psql_models import UserMeeting, UTMParams, ShareLink
from sqlalchemy import delete, or_


In [6]:
async def remove_incorrect_user(incorrect_user_id: str):
    
    async with async_session() as session:
        try:
            # First delete associated tokens
            await session.execute(
                delete(UserToken).where(UserToken.user_id == incorrect_user_id)
            )
            
            # Then delete user meetings
            await session.execute(
                delete(UserMeeting).where(UserMeeting.user_id == incorrect_user_id)
            )
            
            # Delete UTM params if any
            await session.execute(
                delete(UTMParams).where(UTMParams.user_id == incorrect_user_id)
            )
            
            # Delete share links if any
            await session.execute(
                delete(ShareLink).where(
                    or_(
                        ShareLink.owner_id == incorrect_user_id,
                        ShareLink.accepted_by == incorrect_user_id
                    )
                )
            )
            
            # Finally delete the user
            await session.execute(
                delete(User).where(User.id == incorrect_user_id)
            )
            
            await session.commit()
            print(f"Successfully removed user {incorrect_user_id} and all related records")
            
        except Exception as e:
            await session.rollback()
            print(f"Error removing user: {str(e)}")
            raise

# Run in notebook:
await remove_incorrect_user('7a718ecd-84c8-497f-adcf-dad952746c45')

Successfully removed user 7a718ecd-84c8-497f-adcf-dad952746c45 and all related records


In [7]:
from sqlalchemy import select

In [10]:
async def remove_user_by_token(token: str):
    async with async_session() as session:
        try:
            # First find the user_id from the token
            user_token = await session.execute(
                select(UserToken).where(UserToken.token == token)
            )
            user_token = user_token.scalar_one_or_none()
            
            if not user_token:
                print(f"No user found with token {token}")
                return
                
            user_id = user_token.user_id
            
            # Delete all associated records
            await session.execute(delete(UserToken).where(UserToken.user_id == user_id))
            await session.execute(delete(UserMeeting).where(UserMeeting.user_id == user_id))
            await session.execute(delete(UTMParams).where(UTMParams.user_id == user_id))
            await session.execute(
                delete(ShareLink).where(
                    or_(ShareLink.owner_id == user_id, ShareLink.accepted_by == user_id)
                )
            )
            await session.execute(delete(User).where(User.id == user_id))
            
            await session.commit()
            print(f"Successfully removed user {user_id} and all related records")
            
        except Exception as e:
            await session.rollback()
            print(f"Error removing user: {str(e)}")
            raise

# Run in notebook:
await remove_user_by_token('73da9daba9444703b0b71d055f790b6a')

No user found with token 73da9daba9444703b0b71d055f790b6a
